<a href="https://colab.research.google.com/github/cristianmunoz1/ai4eng-proyecto/blob/main/04%20-%20preprocesado%20con%20departamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Replicamos los recursos locales en el entorno virtual

### Descargamos los datos de la competencia al entorno de trabajo

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

 60% 12.0M/20.1M [00:00<00:00, 124MB/s]
100% 20.1M/20.1M [00:00<00:00, 155MB/s]


### Descomprimimos el archivo

In [ ]:
!unzip udea*.zip > /dev/null

In [ ]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  50135751 test.csv
   692501  10666231 118025055 train.csv
  1286075  15528571 172877479 total


## Importe de librerías necesarias

Se importan las librerías necesarias tales como:

- Pandas: Para la manipulación y análisis de datos con los DataFrames.
- Numpy: Para los cálculos matemáticos y el manejo eficiente de los arrays.
- Unidecode: Para normalizar el texto eliminado caracteres especiales o acentos.
- sklearn.preprocessing: Para convertir datos categóricos en un formato que nos sea de mayor utilidad.


In [ ]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 12.6 MB/s eta 0:00:00


In [ ]:
import unidecode as ud
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

### Cargamos el csv como un dataframe de Pandas
Guardamos el cvs en la variable train e imprimimos la cantidad de filas y columnas del DataFrame train.

In [ ]:
train = pd.read_csv("train.csv")
print ("shape of loaded dataframe", train.shape)

shape of loaded dataframe (692500, 12)


Visualizamos las columnas del DataFrame train.

In [ ]:
train.columns

Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE', 'ESTU_PAGOMATRICULAPROPIO',
       'RENDIMIENTO_GLOBAL'],
      dtype='object')

Generamos una vista inicial de los datos.

In [ ]:
train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo


## One_hot encoding para la columna 'ESTU_PRGMACADEMICO'

#### Estrategia de preprocesado de 'ESTU_PRGM_ACADEMICO'

- Observamos cuantas carreras únicas hay y cuales de estas son.
- Las pasamos todas a minúsculas para que se agrupen algunas que eran las mismas pero escritas con diferentes combinaciones de mayúsculas y minúsculas.
- Dejamos en un array de numpy solo las carreras que tengan más de 6000 apariciones en el Dataframe original. Con el objetivo de agrupar al rededor de 20 carreras diferentes.
- A las demás carreras las agrupamos en un grupo 'otros' con el objetivo de reducir el ruido en los datos y la posible variabilidad que puedan presentar.
- Aplicamos OneHotEncoding a los nuevos datos en la columna.

In [ ]:
carreras_unicas = train['ESTU_PRGM_ACADEMICO'].unique()

In [ ]:
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].apply(lambda x: ud.unidecode(x.lower()))

In [ ]:
train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,enfermeria,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,derecho,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,mercadeo y publicidad,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,administracion de empresas,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,psicologia,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo


In [ ]:
counts = train['ESTU_PRGM_ACADEMICO'].value_counts().to_frame()
prgm_acads = (counts.loc[counts['count'] > 6000]).index
prgm_acads


Index(['administracion de empresas', 'derecho', 'contaduria publica',
       'psicologia', 'ingenieria industrial', 'ingenieria civil', 'medicina',
       'ingenieria de sistemas', 'licenciatura en pedagogia infantil',
       'ingenieria ambiental', 'trabajo social', 'enfermeria', 'arquitectura',
       'administracion en salud ocupacional', 'negocios internacionales',
       'economia', 'ingenieria mecanica', 'ingenieria electronica',
       'comunicacion social', 'administracion financiera'],
      dtype='object', name='ESTU_PRGM_ACADEMICO')

In [ ]:
len(prgm_acads)

20

In [ ]:
array = [element for element in train['ESTU_PRGM_ACADEMICO'] if element not in prgm_acads ]
array

condition = train['ESTU_PRGM_ACADEMICO'].isin(array)
train.loc[condition, 'ESTU_PRGM_ACADEMICO'] = "otros"
train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,enfermeria,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,derecho,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,otros,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,administracion de empresas,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,psicologia,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo


In [ ]:
train['ESTU_PRGM_ACADEMICO'].value_counts()

,count
ESTU_PRGM_ACADEMICO,
otros,251299
administracion de empresas,76679
derecho,53244
contaduria publica,51846
psicologia,49309
ingenieria industrial,34794
ingenieria civil,20943
medicina,18099
ingenieria de sistemas,18028


In [ ]:
encode = OneHotEncoder()

codificadorPrg = encode.fit_transform(train[['ESTU_PRGM_ACADEMICO']])

arreglo = pd.DataFrame(codificadorPrg.toarray(), columns = encode.categories_)
print(arreglo)
train.drop('ESTU_PRGM_ACADEMICO', axis = 1)

train = pd.concat([train, arreglo], axis = 'columns')
train['ESTU_PRGM_ACADEMICO']

       administracion de empresas administracion en salud ocupacional  \
0                             0.0                                 0.0   
1                             0.0                                 0.0   
2                             0.0                                 0.0   
3                             1.0                                 0.0   
4                             0.0                                 0.0   
...                           ...                                 ...   
692495                        0.0                                 0.0   
692496                        0.0                                 0.0   
692497                        0.0                                 1.0   
692498                        0.0                                 0.0   
692499                        0.0                                 0.0   

       administracion financiera arquitectura comunicacion social  \
0                            0.0          0.0         

,ESTU_PRGM_ACADEMICO
0,enfermeria
1,derecho
2,otros
3,administracion de empresas
4,psicologia
...,...
692495,otros
692496,psicologia
692497,administracion en salud ocupacional
692498,psicologia


In [ ]:
train = train.drop('ESTU_PRGM_ACADEMICO', axis = 1)

In [ ]:
train.head()

,ID,PERIODO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,...,"(ingenieria de sistemas,)","(ingenieria electronica,)","(ingenieria industrial,)","(ingenieria mecanica,)","(licenciatura en pedagogia infantil,)","(medicina,)","(negocios internacionales,)","(otros,)","(psicologia,)","(trabajo social,)"
0,904256,20212,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,645256,20212,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,308367,20203,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,470353,20195,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,989032,20212,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Categorización ordinal de las columnas categóricas ordinales

Decidimos convertir las variables categóricas que tienen un orden natural del rendimiento global en un formato número para facilitar su procesamiento.

Realizamos una inspección rápida para asegurarnos de que estamos manipulando los datos correctamente.

In [ ]:
train['RENDIMIENTO_GLOBAL'].head()

,RENDIMIENTO_GLOBAL
0,medio-alto
1,bajo
2,bajo
3,alto
4,medio-bajo


In [ ]:
train.columns

Index([                                    'ID',
                                      'PERIODO',
                       'ESTU_PRGM_DEPARTAMENTO',
               'ESTU_VALORMATRICULAUNIVERSIDAD',
                      'ESTU_HORASSEMANATRABAJA',
                         'FAMI_ESTRATOVIVIENDA',
                           'FAMI_TIENEINTERNET',
                          'FAMI_EDUCACIONPADRE',
                          'FAMI_EDUCACIONMADRE',
                     'ESTU_PAGOMATRICULAPROPIO',
                           'RENDIMIENTO_GLOBAL',
                ('administracion de empresas',),
       ('administracion en salud ocupacional',),
                 ('administracion financiera',),
                              ('arquitectura',),
                       ('comunicacion social',),
                        ('contaduria publica',),
                                   ('derecho',),
                                  ('economia',),
                                ('enfermeria',),
                    

## Quitarle los valores nulos a todas las columnas categóricas ordinales


### Conteo de valores nulos

Realizamos un conteo para saber cuantos valores nulos hay en algunas columnas específicas:

In [ ]:
train['FAMI_EDUCACIONMADRE'].value_counts()

,count
FAMI_EDUCACIONMADRE,
7.0,140877
4.0,98936
9.0,88925
11.0,84353
6.0,80557
5.0,55836
12.0,45621
8.0,27330
10.0,22265


In [ ]:
print(f"Cantidad nulos valor matrícula = {train['ESTU_VALORMATRICULAUNIVERSIDAD'].isna().sum()}")
print(f"Cantidad nulos horas semanales trabaja = {train['ESTU_HORASSEMANATRABAJA'].isna().sum()}")
print(f"Cantidad nulos estrato vivienda = {train['FAMI_ESTRATOVIVIENDA'].isna().sum()}")
print(f"Cantidad nulos educación padre = {train['FAMI_EDUCACIONPADRE'].isna().sum()}")
print(f"Cantidad nulos educación madre = {train['FAMI_EDUCACIONMADRE'].isna().sum()}")
print(f"Cantidad nulos rendimiento global = {train['RENDIMIENTO_GLOBAL'].isna().sum()}")

Cantidad nulos valor matrícula = 0
Cantidad nulos horas semanales trabaja = 0
Cantidad nulos estrato vivienda = 0
Cantidad nulos educación padre = 0
Cantidad nulos educación madre = 0
Cantidad nulos rendimiento global = 0


### Cambio de valores nulos por 'no info'

Realizamos este proceso con el fin de no perder información, indicar ausencia de datos y evitar sesgos conservando la distinción de que falta información.

In [ ]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'] = train['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna("no info")
train['ESTU_HORASSEMANATRABAJA'] = train['ESTU_HORASSEMANATRABAJA'].fillna("no info")
train['FAMI_ESTRATOVIVIENDA'] = train['FAMI_ESTRATOVIVIENDA'].fillna("no info")
train['FAMI_EDUCACIONPADRE'] = train['FAMI_EDUCACIONPADRE'].fillna("no info")
train['FAMI_EDUCACIONMADRE'] = train['FAMI_EDUCACIONMADRE'].fillna("no info")
train['RENDIMIENTO_GLOBAL'] = train['RENDIMIENTO_GLOBAL'].fillna("no info")

## Codificación de valores categóricos ordinales.

Se define un conjunto de categorías para cada una de las columnas categóricas ordinales en el DataFrame con un orden lógoc para transformar las columnas en valores numéricos, lo cuál es necesario para realizar nuestro modelo.

In [ ]:
categories = [
    ['no info', 'No pagó matrícula', 'Menos de 500 mil', 'Entre 500 mil y menos de 1 millón',
     'Entre 1 millón y menos de 2.5 millones', 'Entre 2.5 millones y menos de 4 millones',
     'Entre 4 millones y menos de 5.5 millones', 'Entre 5.5 millones y menos de 7 millones',
     'Más de 7 millones'],  # ESTU_VALORMATRICULAUNIVERSIDAD
    ['no info', '0', 'Menos de 10 horas', 'Entre 11 y 20 horas', 'Entre 21 y 30 horas', 'Más de 30 horas'],  # ESTU_HORASSEMANATRABAJA
    ['no info', 'Sin Estrato', 'Estrato 1', 'Estrato 2', 'Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 6'],  # FAMI_ESTRATOVIVIENDA
    ['no info', 'Ninguno', 'No sabe', 'No Aplica', 'Primaria incompleta', 'Primaria completa',
     'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa', 'Técnica o tecnológica incompleta',
     'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa', 'Postgrado'],  # FAMI_EDUCACIONPADRE
    ['no info', 'Ninguno', 'No sabe', 'No Aplica', 'Primaria incompleta', 'Primaria completa',
     'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa', 'Técnica o tecnológica incompleta',
     'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa', 'Postgrado'],  # FAMI_EDUCACIONMADRE
    ['no info', 'bajo', 'medio-bajo', 'medio-alto', 'alto']  # RENDIMIENTO_GLOBAL
]



Utilizamos OrdinalEncoder de Scikitlearn para que a cada categoría dentro de la columna se le asigne un valor numperivo basado en el orden que se definió en el paso anterior.

In [ ]:
codificador = OrdinalEncoder(categories=categories)
train[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL']] = codificador.fit_transform(
    train[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
           'FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONPADRE',
           'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL']]
       )

In [ ]:
train.head()

,ID,PERIODO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,...,"(ingenieria de sistemas,)","(ingenieria electronica,)","(ingenieria industrial,)","(ingenieria mecanica,)","(licenciatura en pedagogia infantil,)","(medicina,)","(negocios internacionales,)","(otros,)","(psicologia,)","(trabajo social,)"
0,904256,20212,BOGOTÁ,7.0,2.0,4.0,Si,8.0,12.0,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,645256,20212,ATLANTICO,5.0,1.0,4.0,No,9.0,8.0,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,308367,20203,BOGOTÁ,5.0,5.0,4.0,Si,7.0,7.0,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,470353,20195,SANTANDER,6.0,1.0,5.0,Si,2.0,7.0,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,989032,20212,ANTIOQUIA,5.0,4.0,4.0,Si,5.0,5.0,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Reemplazamos los valores nulos de la columna con el valor no info.

In [ ]:
train['ESTU_PAGOMATRICULAPROPIO'] = train['ESTU_PAGOMATRICULAPROPIO'].fillna('no info')
train['ESTU_PAGOMATRICULAPROPIO'].value_counts()

,count
ESTU_PAGOMATRICULAPROPIO,
No,382201
Si,303801
no info,6498


Eliminamos todas las filas donde la columna ESTU_PAGOMATRICULAPROPIO tenga el valor de no info para que el modelo no se vea afectado por filas que no tienen información

In [ ]:
train = train[train['ESTU_PAGOMATRICULAPROPIO'] != 'no info']

In [ ]:
train['ESTU_PAGOMATRICULAPROPIO'].value_counts()

,count
ESTU_PAGOMATRICULAPROPIO,
No,382201
Si,303801


In [ ]:
print(f"Cantidad nulos tiene internet = {train['ESTU_PAGOMATRICULAPROPIO'].isna().sum()}")
train['FAMI_TIENEINTERNET'].value_counts()

Cantidad nulos tiene internet = 0


,count
FAMI_TIENEINTERNET,
Si,588047
No,73033


## Conversión One-Hot de las columnas FAMI_TIENEINTERNET y ESTU_PAGOMATRICULAPROPIO

Preparamos los datos para hacer el uso del modelo con un one-hot.

In [ ]:
one_hot_encoded = pd.get_dummies(train['ESTU_PAGOMATRICULAPROPIO'], prefix = 'ESTU_PAGOMATRICULAPROPIO')
one_hot_encoded = one_hot_encoded.astype(int)
one_hot_encoded

,ESTU_PAGOMATRICULAPROPIO_No,ESTU_PAGOMATRICULAPROPIO_Si
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
692495,0,1
692496,1,0
692497,0,1
692498,0,1


In [ ]:
train = pd.concat([train, one_hot_encoded], axis = 1)
train = train.drop('ESTU_PAGOMATRICULAPROPIO', axis = 1)

In [ ]:
train['ESTU_PAGOMATRICULAPROPIO_No'].head()

,ESTU_PAGOMATRICULAPROPIO_No
0,1
1,1
2,1
3,1
4,1


In [ ]:
print(train.columns)

Index([                                    'ID',
                                      'PERIODO',
                       'ESTU_PRGM_DEPARTAMENTO',
               'ESTU_VALORMATRICULAUNIVERSIDAD',
                      'ESTU_HORASSEMANATRABAJA',
                         'FAMI_ESTRATOVIVIENDA',
                           'FAMI_TIENEINTERNET',
                          'FAMI_EDUCACIONPADRE',
                          'FAMI_EDUCACIONMADRE',
                           'RENDIMIENTO_GLOBAL',
                ('administracion de empresas',),
       ('administracion en salud ocupacional',),
                 ('administracion financiera',),
                              ('arquitectura',),
                       ('comunicacion social',),
                        ('contaduria publica',),
                                   ('derecho',),
                                  ('economia',),
                                ('enfermeria',),
                      ('ingenieria ambiental',),
                    

In [ ]:
one_hot_encoded = pd.get_dummies(train['FAMI_TIENEINTERNET'], prefix = 'FAMI_TIENEINTERNET')
one_hot_encoded = one_hot_encoded.astype(int)
one_hot_encoded

,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si
0,0,1
1,1,0
2,0,1
3,0,1
4,0,1
...,...,...
692495,0,1
692496,0,1
692497,0,1
692498,1,0


Concatenamos las columnas One-hot al DataFrame original y eliminamos la columna original ya que no es necesaria, y así evitamos redundancia en el conjunto de datos

In [ ]:
train = pd.concat([train, one_hot_encoded], axis = 1)
train = train.drop('FAMI_TIENEINTERNET', axis = 1)

Finalmente, realizamos una inspeccion y revision de las nuevas columnas generadas.

In [ ]:
train['FAMI_TIENEINTERNET_Si'].head()

,FAMI_TIENEINTERNET_Si
0,1
1,0
2,1
3,1
4,1


### Agrupamos los departamentos entre los que tienen más o menos recursos

Nuestro objetivo es agrupar los departamentos por la incidencia de la pobreza monetaria en x departamento. Esto dado que pensamos que la incidencia de la pobreza es un factor fundamental para el desarrollo académico de las personas en el país.

In [ ]:
np.unique(train['ESTU_PRGM_DEPARTAMENTO'].values)

array(['AMAZONAS', 'ANTIOQUIA', 'ARAUCA', 'ATLANTICO', 'BOGOTÁ',
       'BOLIVAR', 'BOYACA', 'CALDAS', 'CAQUETA', 'CASANARE', 'CAUCA',
       'CESAR', 'CHOCO', 'CORDOBA', 'CUNDINAMARCA', 'GUAVIARE', 'HUILA',
       'LA GUAJIRA', 'MAGDALENA', 'META', 'NARIÑO', 'NORTE SANTANDER',
       'PUTUMAYO', 'QUINDIO', 'RISARALDA', 'SAN ANDRES', 'SANTANDER',
       'SUCRE', 'TOLIMA', 'VALLE', 'VAUPES'], dtype=object)

In [ ]:
richest_deps = ['BOGOTÁ', 'RISARALDA', 'SANTANDER', 'ANTIOQUIA', 'CALDAS', 'CUNDINAMARCA', 'META', 'VALLE', 'QUINDIO', 'ATLANTICO', 'BOYACA']

In [ ]:
train['ESTU_PRGM_DEPARTAMENTO'] = train['ESTU_PRGM_DEPARTAMENTO'].apply(lambda x: 1 if x in richest_deps else 0)

In [ ]:
train['ESTU_PRGM_DEPARTAMENTO'].values

array([1, 1, 1, ..., 1, 0, 1])

In [ ]:
train.columns

Index([                                    'ID',
                                      'PERIODO',
                       'ESTU_PRGM_DEPARTAMENTO',
               'ESTU_VALORMATRICULAUNIVERSIDAD',
                      'ESTU_HORASSEMANATRABAJA',
                         'FAMI_ESTRATOVIVIENDA',
                          'FAMI_EDUCACIONPADRE',
                          'FAMI_EDUCACIONMADRE',
                           'RENDIMIENTO_GLOBAL',
                ('administracion de empresas',),
       ('administracion en salud ocupacional',),
                 ('administracion financiera',),
                              ('arquitectura',),
                       ('comunicacion social',),
                        ('contaduria publica',),
                                   ('derecho',),
                                  ('economia',),
                                ('enfermeria',),
                      ('ingenieria ambiental',),
                          ('ingenieria civil',),
                    

### Eliminamos la columna periodo y ID ya que se considera, no tiene efecto sobre el rendimiento

In [ ]:
train = train.drop('PERIODO', axis = 1)
train = train.drop('ID', axis = 1)

In [ ]:
train.head()

,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,"(administracion de empresas,)","(administracion en salud ocupacional,)","(administracion financiera,)",...,"(licenciatura en pedagogia infantil,)","(medicina,)","(negocios internacionales,)","(otros,)","(psicologia,)","(trabajo social,)",ESTU_PAGOMATRICULAPROPIO_No,ESTU_PAGOMATRICULAPROPIO_Si,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si
0,1,7.0,2.0,4.0,8.0,12.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1
1,1,5.0,1.0,4.0,9.0,8.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,0
2,1,5.0,5.0,4.0,7.0,7.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1,0,0,1
3,1,6.0,1.0,5.0,2.0,7.0,4.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1
4,1,5.0,4.0,4.0,5.0,5.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1,0,0,1


In [ ]:
train.to_csv('train2.csv', index = False)

In [ ]:
!head train2.csv

ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,"('administracion de empresas',)","('administracion en salud ocupacional',)","('administracion financiera',)","('arquitectura',)","('comunicacion social',)","('contaduria publica',)","('derecho',)","('economia',)","('enfermeria',)","('ingenieria ambiental',)","('ingenieria civil',)","('ingenieria de sistemas',)","('ingenieria electronica',)","('ingenieria industrial',)","('ingenieria mecanica',)","('licenciatura en pedagogia infantil',)","('medicina',)","('negocios internacionales',)","('otros',)","('psicologia',)","('trabajo social',)",ESTU_PAGOMATRICULAPROPIO_No,ESTU_PAGOMATRICULAPROPIO_Si,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si
1,7.0,2.0,4.0,8.0,12.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,1
1,5.0,1.0,4.0,9.0,8.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.